# merge segmentations UKB, KORA, NAKO


In [1]:
import os

import nibabel  # for loading and saving nifty files
import torch
from math import ceil, floor
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Pad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from nibabel.affines import from_matvec, to_matvec, apply_affine
import nilearn

In [2]:
print(os.getcwd())

/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets


## combination of segmentation files:
creates new segmentation image of the same size as the merged volume  
changes the labels of the segmentation maps according to label list below  
also reads header file of .nrrd files.  
merges segmentations according to header['space origin']


In [3]:
def combine_seg(seg_files, data_dir):
    #data_dir = seg_dir
    img_file = [f for f in seg_files if 'input_scan.nii.gz' in f and not 'combined' in f]
    print('img file: ', img_file)
    if len(img_file) == 1:
        img_file = img_file[0]
    else:
        print('less or more than 1 img file found')

    print('data_dir ', data_dir)
    print('img_file ', img_file)
    
    print('load and reorient image')
    img, data, img_h = reorient_image(os.path.join(data_dir, img_file), is_label=False)
    
    #ONLY FOR KORA!!!!
    #data = np.flip(np.moveaxis(data, 2, 1))  # 012 -> 021
    
    #img = nibabel.load(os.path.join(data_dir, img_file))
    #print(img)
    #img_h = img.header
    #print(img_h)
    print('x: ', img_h['qoffset_x'])
    print('y: ', img_h['qoffset_y'])
    print('z: ', img_h['qoffset_z'])

    # somehow this is wrong. x, and y should be positive and z negative

    segmentations = {'liver': [],
                    'spleen': [],
                    'kidney_r': [],
                    'kidney_l': [],
                    'adrenal_l': [],
                    'adrenal_r': [],
                    'pancreas': [],
                    'gallbladder': []}
    for file in seg_files:
        if "nrrd" in file:
            print('load and reorient segmentation')
            seg, seg_data, header = reorient_image(os.path.join(data_dir, file), is_label=True)
            
            #seg, header = nrrd.read(os.path.join(data_dir, file))

            # change labels according to label list above. liver stays the same, as it gets label=1         
            seg, seg_label = change_label(seg_data, file)

            # print out some important header information:
            ##print(file)
            
            print('segm shape: ', seg_data.shape)
            print('x: ', header['qoffset_x'])
            print('y: ', header['qoffset_y'])
            print('z: ', header['qoffset_z'])
            
            #space_directions = header['space directions']
            #space_origin = header['space origin']
            #sizes = header['sizes']
            #print('z: ', space_origin[2])
            #print('size z: ', sizes[2])
            #print('spacing z: ', space_directions[2,2])

            segmentations[seg_label] = [seg_data,header] # append tuple of seg and header, need header later
    #print(segmentations)

    # check for overlaps:
    try:
        s1, s2 = check_for_overlaps([segmentations['liver'][0], 'liver'], [segmentations['gallbladder'][0], 'gallbladder'])
        segmentations['liver'][0] = s1
        segmentations['gallbladder'][0] = s2
    except IndexError as err :
        print('doesnt have one of the segmentations liver or gallbladder')
    except ValueError as err:
        print('liver and gallbladder sizes differ')
    
    
    #print(segmentations)
    try:
        s1, s2 = check_for_overlaps([segmentations['kidney_l'][0], 'kidney_l'], [segmentations['spleen'][0], 'spleen'])
   
        segmentations['kidney_l'][0] = s1
        segmentations['spleen'][0] = s2
    except IndexError as err: 
        print('doesnt have one of the segmentations kidney l or spleen')
    except ValueError as err:
        print('spleen and kidney sizes differ')

    
    #s1, s2 = check_for_overlaps([segmentations['kidney_l'][0], 'kidney_l'], [segmentations['pancreas'][0], 'pancreas'])
    #segmentations['kidney_l'][0] = s1
    #segmentations['pancreas'][0] = s2
    
    ##s1, s2 = check_for_overlaps([segmentations['spleen'][0], 'spleen'], [segmentations['pancreas'][0], 'pancreas'])
    #segmentations['spleen'][0] = s1
    #segmentations['pancreas'][0] = s2
    
    try:
        s1, s2 = check_for_overlaps([segmentations['adrenal_r'][0], 'adrenal_r'], [segmentations['gallbladder'][0], 'gallbladder'])
        segmentations['adrenal_r'][0] = s1
        segmentations['gallbladder'][0] = s2
    except IndexError as err:
        print('doesnt have one of the segmentations adrenal r or gallbladder')
    except ValueError as err:
        print('adrenal and gallbladder sizes differ')
    
    try:
        s1, s2 = check_for_overlaps([segmentations['adrenal_l'][0], 'adrenal_l'], [segmentations['pancreas'][0], 'pancreas'])
        segmentations['adrenal_l'][0] = s1
        segmentations['pancreas'][0] = s2
    except IndexError as err:
        print('doesnt have one of the segmentations adrenal l or pancreas')
    except ValueError as err:
        print('adrenal and pancreas sizes differ')
        
    #print(len(segmentations))
    num_segm = len(segmentations)
    if num_segm < 8:
        print('less than 8 segmentations')

    #segm_sorted = sorted(segmentations, key=lambda seg: seg[1]['space origin'][2], reverse=True)

    comb_segm = np.zeros_like(img.get_data())
    print('final shape: ', comb_segm.shape)

    img_dim_v = img.shape

    
    for label in segmentations:
        print(label)
        
        #print('reorient seg')
        #seg, segm, header = reorient_image(seg, is_label=True)
        try: 
            segm = segmentations[label][0]
            header = segmentations[label][1]
        except IndexError as ierr:
            print('doesnt have the segmentation for ', label)
            continue
        #print(segm)
        #print(header)
        
        #segm_off = header['space origin']
        #segm_dim_v = header['sizes']
        #segm_spacing = abs(np.diag(header['space directions']))
        
        segm_spacing = abs(header['pixdim'][1:4])
        segm_dim_v = segm.shape
        # CAVE: not sure if abs is correct here
        segm_offx = abs(header['qoffset_x'])
        segm_offy = abs(header['qoffset_y'])
        segm_offz = header['qoffset_z']


        segm_offset = np.array([segm_offx, segm_offy, segm_offz])
        # if same size, just add the segmentation
        print('im dim v: ', img_dim_v)
        print('segm dim v: ', segm_dim_v)
            
            
        if (segm_dim_v == img_dim_v):
            print('same')
            comb_segm += segm
        else:
            print('im dim v: ', img_dim_v)
            print('segm dim v: ', segm_dim_v)

            im_spacing = abs(img_h['pixdim'][1:4])
            print('im spacing: ', im_spacing)
            print('segm spacing: ', segm_spacing)

            im_dim_w = img_dim_v * im_spacing
            segm_dim_w = segm_dim_v * segm_spacing

            print('im dim w: ', im_dim_w)
            print('segm dim w: ', segm_dim_w)

            # correction of the wrong information in header file
            im_offx = abs(img_h['qoffset_x'])
            im_offy = abs(img_h['qoffset_y'])
            im_offz = img_h['qoffset_z']


            im_offset = np.array([im_offx, im_offy, im_offz])
            im_start = im_offset
            im_end = np.array([im_start[0]-im_dim_w[0], im_start[1]-im_dim_w[1], im_start[2]+im_dim_w[2]])

            segm_start = segm_offset
            segm_end = np.array([segm_start[0]-segm_dim_w[0], segm_start[1]-segm_dim_w[1], segm_start[2]+segm_dim_w[2]])

            print('im off: ', im_offset)
            print('segm off: ', segm_offset)

            print('im start: ', im_start)
            print('im end: ', im_end)
            print('segm start: ', segm_start)
            print('segm end: ', segm_end)

            start_diff_w = abs(im_start - segm_start)
            end_diff_w = abs(im_end - segm_end)
            print('start diff w: ', start_diff_w)

            print('end_diff w ', end_diff_w)

            start_diff_v = np.round(start_diff_w / segm_spacing).astype(int)
            print('start diff v: ', start_diff_v)

            end_diff_v = np.round(end_diff_w / segm_spacing).astype(int)
            print('end diff v: ', end_diff_v)

            segm_end_v = img_dim_v - end_diff_v
            print('segm end v: ', segm_end_v)

            segm_end_x = segm_end_v[0]
            segm_end_y = segm_end_v[1]
            segm_end_z = segm_end_v[2]
            segm_start_x = segm_end_x - segm_dim_v[0]
            segm_start_y = segm_end_y - segm_dim_v[1]
            segm_start_z = segm_end_z - segm_dim_v[2]
            print('segm start v: ', segm_start_x, segm_start_y, segm_start_z)

            comb_segm[segm_start_x:segm_end_x, segm_start_y:segm_end_y, segm_start_z:segm_end_z] += segm

    empty_header = nibabel.Nifti1Header()
    new_img_nii = nibabel.Nifti1Image(comb_segm, img.affine, img_h)
    #new_img_nii.header['pixdim'] = img_h['pixdim']

    #print(new_img_nii.header)
    new_file_path = os.path.join(data_dir,'combined_segmentation.nii.gz')
    nibabel.save(new_img_nii, new_file_path)
    
    # also save reoriented image:
    new_file_path = os.path.join(data_dir,'input_scan.nii.gz')
    
    nibabel.save(img, new_file_path)



### changes labels according to this list:
liver = 1  
spleen = 2  
kidney_r = 3  
kidney_l = 4  
adrenal_r = 5  
adrenal_l = 6  
pancreas = 7  
gallbladder = 8

Caution: does not consider spelling mistakes ;) have to manually check first if all files are named correctly


In [4]:
def change_label(seg, f):
    print(f)
    seg_label = None
    if 'liver' in f or 'Liver' in f:
        print('found liver')
        seg[seg == 1] = 1
        seg_label = 'liver'
        
    elif 'spleen' in f or 'Spleen' in f:
        print('found spleen')
        seg[seg == 1] = 2
        seg_label = 'spleen'
    elif ('kidney' in f or 'Kidney' in f) and ('right' in f  or 'Right' in f):
        seg[seg==1] = 3
        print('found right kidney')
        seg_label = 'kidney_r'
        
    elif ('kidney' in f or 'Kidney' in f) and ('left' in f  or 'Left' in f): 
        seg[seg==1] = 4
        print('found left kidney')
        seg_label = 'kidney_l'

    
    elif ('adrenal' in f or 'Adrenal' in f) and ('right' in f  or 'Right' in f): 
        seg[seg==1] = 5
        print('found right adrenal')
        seg_label = 'adrenal_r'
        
    elif ('adrenal' in f or 'Adrenal' in f) and ('left' in f  or 'Left' in f):
        seg[seg==1] = 6
        print('found left adrenal')
        seg_label = 'adrenal_l'
        
    elif ('pancreas' in f or 'Pancreas' in f):
        seg[seg==1] = 7
        print('found pancreas')
        seg_label = 'pancreas'
    
    elif ('gallbladder' in f or 'Gallbladder' in f):
        seg[seg==1] = 8
        print('found gallbladder')
        seg_label = 'gallbladder'
    else:
        seg[seg==1] = 0
        print('found other organ, setting labels to 0')
        
    return seg, seg_label

## checks for overlaps of segmentations

even though Oksana checked manually if some segmentations overlap, there might be some few voxels that still overlap.

This function just checks the organs that have highest risk of overlapping ;)  

main overlaps to be expected between gallbladder and liver  

CAVE: this only works if the segmentations have the same shape... (so far I didn't encounter any overlaps when the segmentation shapes mismatched.. but of course this should be handled better)

In [5]:
def check_for_overlaps(seg1, seg2):
    seg1, seg1_label = seg1
    seg2, seg2_label = seg2
    print('seg 1 shape', seg1.shape)
    print('seg 2 shape', seg2.shape)
    print(seg1_label, seg2_label)
    
    if seg1_label == 'liver' and seg2_label == 'gallbladder':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 8)
        print(overlap)
        seg1[overlap] = 0
        
    if seg1_label == 'kidney_l' and seg2_label == 'spleen':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 8)
        print(overlap)
        seg1[overlap] = 0
        
    if seg1_label == 'adrenal_r' and seg2_label == 'gallbladder':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 24)
        print(overlap)
        seg1[overlap] = 0
    
    if seg1_label == 'adrenal_l' and seg2_label == 'pancreas':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 42)
        print(overlap)
        seg1[overlap] = 0
    return seg1, seg2
        

In [6]:
def reorient_image(in_file, axes='RAS', translation_params=(0, 0, 0), is_label=False):
    # Global parameters
    POSSIBLE_AXES_ORIENTATIONS = [
        "LAI", "LIA", "ALI", "AIL", "ILA", "IAL",
        "LAS", "LSA", "ALS", "ASL", "SLA", "SAL",
        "LPI", "LIP", "PLI", "PIL", "ILP", "IPL",
        "LPS", "LSP", "PLS", "PSL", "SLP", "SPL",
        "RAI", "RIA", "ARI", "AIR", "IRA", "IAR",
        "RAS", "RSA", "ARS", "ASR", "SRA", "SAR",
        "RPI", "RIP", "PRI", "PIR", "IRP", "IPR",
        "RPS", "RSP", "PRS", "PSR", "SRP", "SPR"
    ]

    #Load the image to rectify
    if in_file.split('.')[-1] == 'nrrd':
        print('NRRD file received, converting to nifti!!!')
        image = nrrd_to_nifti(in_file)
    else:
        image = nibabel.load(in_file)
        print(image.get_affine())
    #if not is_label:
        #volid = in_file.split('/')[-2]
        #PreProcess.save_nibabel(volume=image.get_fdata(), header=image.header, filename=f'{volid}_vol_after_loaded')

    # Get the axes automatically.

    header = image.header
    #print(header)    
    axes = nibabel.orientations.aff2axcodes(header.get_best_affine())
    axes = "".join(list(axes))

    # Check that a valid input axes is specified
    if axes not in POSSIBLE_AXES_ORIENTATIONS:
        raise ValueError("Wrong coordinate system: {0}.".format(axes))

    rotation = swap_affine(axes)
    det = np.linalg.det(rotation)
    if det != 1:
        raise Exception("Rotation matrix determinant must be one "
                        "not '{0}'.".format(det))

    affine = image.get_affine()
    # Get the trnaslation to apply
    translation = np.eye(4)
    # Get the input image affine transform
    # s, t = to_matvec(affine)
    # print(affine, s, t, tuple(t))
    # print(header.get_best_affine())
    # volume_translations = [-216, -178.9, -664.5]
    # current_t = [-119, -106, -503]
    # intended_t = tuple(np.subtract(volume_translations, t))
    # print(intended_t)
    translation[:3, 3] = translation_params  # tuple(t) #if not is_label else translation_params[:3, 3]
    # print(affine, data.shape)

    # source_orientation = nb.orientations.io_orientation(affine)
    # transformation_mat = nb.orientations.ornt_transform(source_orientation, PreProcess.get_target_orientation())
    # data = nb.orientations.apply_orientation(data, transformation_mat)
    # print(data.shape)

    # Apply the rotation to set the image in the RAS coordiante system
    transformation = np.dot(np.dot(rotation, affine), translation)
    # print('transformation mat:', transformation)
    # image.set_qform(transformation)
    image.set_sform(transformation)


    data = image.get_data()
    # print('after transformation:', data.shape)
    # affine = image.get_affine()
    # s, t = to_matvec(affine)
    # print(affine, s, t, tuple(t))
    # translation = np.eye(4)
    # translation[:3, 3] = tuple(-t)
    # image.set_sform(translation)
    # data = image.get_data()
    # print('after translation:', data.shape)

    # array = image.get_data()
    # print(image.get_affine(), array.shape)

    header = image.header
    new_axes = nibabel.orientations.aff2axcodes(header.get_best_affine())
    new_axes = "".join(list(new_axes))
    # 216.09375, 178.90625, -664.5
    print(f"{axes} transformed to {new_axes}")
    # print(data.shape, header.get_best_affine())
    # print('-' * 39)
    # if not is_label:
    #     return image, data, header, translation
    return image, data, header

In [7]:
def swap_affine(axes):
    CORRECTION_MATRIX_COLUMNS = {
        "R": (1, 0, 0),
        "L": (-1, 0, 0),
        "A": (0, 1, 0),
        "P": (0, -1, 0),
        "S": (0, 0, 1),
        "I": (0, 0, -1)
    }

    if axes not in ['RSP', 'LIP', 'RAS', 'LPS']:
        raise Exception(
            f'Unknown axes passed for affine transformation! Please add transformation for {axes} manually.')
    rotation = np.eye(4)
    rotation[:3, 0] = CORRECTION_MATRIX_COLUMNS[axes[0]]
    rotation[:3, 1] = CORRECTION_MATRIX_COLUMNS[axes[1]]
    rotation[:3, 2] = CORRECTION_MATRIX_COLUMNS[axes[2]]
    # print(rotation)
    if axes == "RSP":
        rotation = np.array([[1., 0., 0., 0.],
                             [0., 0., 1., 0.],
                             [0., -1., 0., 0.],
                             [0., 0., 0., 1.]])
        # print(rotation)
    elif axes == "LPS":
        rotation = np.array([
            [-1., 0., 0., 0.],
            [0., -1., 0., 0.],
            [0., 0., 1., 0.],
            [0., 0., 0., 1.]
        ])
        # print(rotation)
    return rotation


In [8]:
def nrrd_to_nifti(file_path):
    #output_file_path = os.path.join(file_path,'test')+'out.nii.gz'

    data, header = nrrd.read(file_path)

    if len(data.shape) == 4:
        data = data[:, :, :, 0]

    affine = header['space directions']
    affine = affine[:3, :3]
    origins = header['space origin']
    origins = origins[:3]
    t_mat = from_matvec(affine, origins)
    img = nibabel.Nifti1Image(data, t_mat)
    #nibabel.save(img, output_file_path)
    #img = nibabel.load(output_file_path)

    return img

## Calls the combine_seg function. 

change data_dir accordingly. Expects all segmentations and the input scan (in case of UKB the combined input file ...comb_01.nii.gz) in .nrrd or .nii.gz format

seg_dir = directory of the segmentation files, also should include the merged volume ...comb_01.nii.gz

In [68]:
data_dir = '/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/data/new/KORA/data'
listFolders =  os.listdir(data_dir)
print('list folders: ', listFolders)

for folder in listFolders:
    print('processing folder: ', folder)
    seg_dir = os.path.join(data_dir,folder)

    listFiles = os.listdir(seg_dir)
    sequence = 'opp'
    seg_files = [f for f in listFiles if ".nrrd" in f or '.nii.gz' in f]
    #print(listFiles)


    combine_seg(seg_files, seg_dir)
    




list folders:  ['2460723', '2460830', '2459728', '2460734', '2456120', '2456289', '2453666', '2458366', '2460408', '2460249', '2459745', '2453363']
processing folder:  2460723
img file:  ['input_scan.nii.gz']
data_dir  /home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/data/new/KORA/data/2460723
img_file  input_scan.nii.gz
load and reorient image
[[   1.70138884    0.            0.         -235.24905396]
 [   0.            1.70138884    0.          -31.96945   ]
 [   0.            0.            1.69999695 -109.47180176]
 [   0.            0.            0.            1.        ]]


/home/anne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/home/anne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


RAS transformed to RAS
x:  235.24905
y:  109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (left).nrrd
found left kidney
segm shape:  (288, 288, 160)
x:  -235.24905
y:  -109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney(right).nrrd
found right kidney
segm shape:  (288, 288, 160)
x:  -235.24905
y:  -109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-26.nrrd_Pancreas.nrrd
found pancreas
segm shape:  (288, 288, 160)
x:  -235.24905
y:  -109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Spleen.nrrd
found spleen

RSP transformed to RAS
2459728_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (Right).nrrd
found right kidney
segm shape:  (288, 288, 160)
x:  -259.62518
y:  -124.04505
z:  -89.33919
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2459728_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Gallbladder.nrrd
found gallbladder
segm shape:  (288, 288, 160)
x:  -259.62518
y:  -124.04505
z:  -89.33919
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2459728_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nrrd_Adrenal gland(right).nrrd
found right adrenal
segm shape:  (288, 288, 160)
x:  -259.62518
y:  -124.04505
z:  -89.33919
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
liver gallbladder
(array([112, 113]), array([191, 191]), array([44, 44]))
doesnt have one of the segmentations kidney l or spleen
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
adrenal_r gallbladder
(array([], dtype=int64), 

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
adrenal_r gallbladder
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
adrenal_l pancreas
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
final shape:  (288, 288, 160)
liver
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
spleen
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
kidney_r
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
kidney_l
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
adrenal_l
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
adrenal_r
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
pancreas
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
gallbladder
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
processing folder:  2456289
img file: 

processing folder:  2458366
img file:  ['input_scan.nii.gz']
data_dir  /home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/data/new/KORA/data/2458366
img_file  input_scan.nii.gz
load and reorient image
[[   1.70138884    0.            0.         -245.01789856]
 [   0.            1.70138884    0.          -34.41421509]
 [   0.            0.            1.69999695 -144.90783691]
 [   0.            0.            0.            1.        ]]
RAS transformed to RAS
x:  245.0179
y:  144.90784
z:  34.414215
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2458366_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (Right).nrrd
found right kidney
segm shape:  (288, 288, 176)
x:  -245.0179
y:  -144.90784
z:  34.414215
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2458366_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Gallbladder.nrrd
found gallbladder
segm shape:  (288

found left adrenal
segm shape:  (288, 288, 160)
x:  -261.92883
y:  -97.33311
z:  -210.32397
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460249_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_Gallbladder.nrrd
found gallbladder
segm shape:  (288, 288, 160)
x:  -261.92883
y:  -97.33311
z:  -210.32397
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460249_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_Liver.nrrd
found liver
segm shape:  (288, 288, 160)
x:  -261.92883
y:  -97.33311
z:  -210.32397
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460249_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_Kidney(left).nrrd
found left kidney
segm shape:  (288, 288, 160)
x:  -261.92883
y:  -97.33311
z:  -210.32397
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460249_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_Spleen.

RSP transformed to RAS
2453363_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii_Liver.nrrd
found liver
segm shape:  (288, 288, 160)
x:  -234.75934
y:  -113.418205
z:  -106.41784
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2453363_t1_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Gallbladder-2.nrrd
found gallbladder
segm shape:  (288, 288, 160)
x:  -234.75934
y:  -113.418205
z:  -106.41784
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2453363_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (Left).nrrd
found left kidney
segm shape:  (288, 288, 160)
x:  -234.75934
y:  -113.418205
z:  -106.41784
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2453363_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (Right).nrrd
found right kidney
segm shape:  (288, 288, 160)
x:  -234.75934
y:  -113.418205
z:  -106.41784
seg 1 shape (288, 288, 160)
seg 2 shape (28

In [57]:
import numpy.linalg as npl
from math import ceil, floor

data_dir = '/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/data/new/missing'
listFolders =  os.listdir(data_dir)
print('list folders: ', listFolders)

for folder in listFolders:
    print('processing folder: ', folder)
    seg_dir = os.path.join(data_dir,folder)

    listFiles = os.listdir(seg_dir)
    sequence = 'opp'
    seg_files = [f for f in listFiles if ".nrrd" in f or '.nii.gz' in f]
    #print(listFiles)


    #combine_seg(seg_files, seg_dir)
    images = []
    segms = []
    # load files
    for file in seg_files:
        #print(file)
        if file.split('.')[-1] == 'nrrd':
            if 'spleen' in file or 'Spleen' in file:
                print('found spleen')
                print('NRRD file received, converting to nifti!!!')
                segm = nrrd_to_nifti(os.path.join(seg_dir, file))
                segms.append(segm)
                segm_h = segm.header
                print(segm.get_affine())
                print('x: ', segm_h['qoffset_x'])
                print('y: ', segm_h['qoffset_y'])
                print('z: ', segm_h['qoffset_z'])
                
                print(segm_h.get_data_shape())
            '''
            elif 'liver' in file or 'Liver' in file:
                print('found liver')
                print('NRRD file received, converting to nifti!!!')
                segm = nrrd_to_nifti(os.path.join(seg_dir, file))
                segms.append(segm)
                segm_h = segm.header
                print(segm.header.get_best_affine())
                print('x: ', segm_h['qoffset_x'])
                print('y: ', segm_h['qoffset_y'])
                print('z: ', segm_h['qoffset_z'])
                print(segm_h.get_data_shape())
            '''

        elif 'input_scan.nii.gz' in file :
            image = nibabel.load(os.path.join(seg_dir, file))
            print(image.header.get_best_affine())
            images.append(image)

            img_h = image.header
            print('x: ', img_h['qoffset_x'])
            print('y: ', img_h['qoffset_y'])
            print('z: ', img_h['qoffset_z'])
            print(img_h.get_data_shape())
            
    
    new_liver = np.zeros_like(image.get_data())
    old_liver = segms[0].get_data()
    img_affine = image.header.get_best_affine()
    c2 = img_affine[:,1].copy()
    c3 = img_affine[:,2].copy()
    #r2 = img_affine[1,:].copy()
    #r3 = img_affine[2,:].copy()
    #img_affine[1,:] = r3
    #img_affine[2,:] = r2
    img_affine[:,1] = c3
    img_affine[:,2] = c2
    print('changed img affine ', img_affine)
    liver_affine = segms[0].header.get_best_affine()
    #liver_affine[0,3] *= -1
    #liver_affine[1,3] *= -1
    
    c2 = liver_affine[:,1].copy()
    
    c3 = liver_affine[:,2].copy()
    liver_affine[:,1] = c3
    liver_affine[:,2] = c2
    liver_affine[0,3] *= -1
    liver_affine[1,3] *=-1
    liver_affine[0,0] *=-1
    liver_affine[2,2] *=-1
    print('changed liver affine ', liver_affine)
    liver_affine_inv =  npl.inv(liver_affine)
    print('liver inv ', liver_affine_inv)
    img_to_liver_vox = liver_affine_inv.dot(img_affine)
    liver_to_img_vox = npl.inv(img_affine).dot(liver_affine)
    print('livertoimgvox ', liver_to_img_vox)
    #segm2 = nilearn.image.resample_img(segms[0], target_affine=img_to_liver_vox, target_shape= image.get_data().shape, interpolation=nearest)
    
    
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            for z in range(image.shape[2]):
                #img_w = nibabel.affines.apply_affine(img_affine, [x,y,z])
                #print('img_w ', img_w)
                segm_v = nibabel.affines.apply_affine(img_to_liver_vox, [x,y,z])
                #print('segm v ', segm_v)
                #print('img ', x,y,z, ' segm ', (segm_v))
                segm_v = [int(round(v)) for v in segm_v]
                #segm_v = segm_v.astype(int)
                #print(segm_v)
                if segm_v[2] >= old_liver.shape[2]:
                    print(segm_v)
                    segm_v[2] = old_liver.shape[2]-1
                if segm_v[0] >= old_liver.shape[0]:
                    print(segm_v)
                    segm_v[0] = old_liver.shape[2]-1
                if segm_v[1] >= old_liver.shape[1]:
                    print(segm_v)
                    segm_v[1] = old_liver.shape[2]-1
                #print(segm_v)
                #print(old_liver.shape)
                new_liver[x,y,z] = old_liver[segm_v[0], segm_v[1], segm_v[2]]
    
                
                
    #segms[0].set_data_dtype(np.float)
    #print(segms[0].get_data_dtype())
    #print(np.max(segms[0].get_data()))
    #segm2 = nilearn.image.resample_img(segms[0], target_affine=liver_to_img_vox, 
                                       #target_shape=image.get_data().shape, interpolation='nearest')
    #print(np.max(segm2.get_data()))
    # try this
    #segm2 = segms[0].get_data()[4:-1,4:292,:-1]
    #print(segm2.shape)
    #new_img_nii = nibabel.MGHImage(segms[0].get_data(), img_h.get_best_affine(), img_h)
    #new_file_path = os.path.join(seg_dir, 'cropped_spleen.nii.gz')
    #nibabel.save(new_img_nii, new_file_path)
    
    #segm2 = segms[1].get_data()[4:-1,4:292,:-1]
    #print(segm2.shape)
    new_img_nii = nibabel.MGHImage(new_liver, img_h.get_best_affine(), img_h)
    new_file_path = os.path.join(seg_dir, 'cropped_spleen2.nii.gz')
    nibabel.save(new_img_nii, new_file_path)
    
    

        

list folders:  ['2456289']
processing folder:  2456289
[[ -1.70138884   0.           0.         235.26948547]
 [  0.           0.          -1.69999695 144.93167114]
 [  0.          -1.70138884   0.         -21.65872765]
 [  0.           0.           0.           1.        ]]
x:  235.26949
y:  144.93167
z:  -21.658728
(288, 288, 176)
found spleen
NRRD file received, converting to nifti!!!


/home/anne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


[[   1.66666663   -0.           -0.         -235.26948547]
 [  -0.           -0.           -1.66666663 -144.93167114]
 [  -0.            1.69999695    0.          -21.65872765]
 [   0.            0.            0.            1.        ]]
x:  -235.26949
y:  -144.93167
z:  -21.658728
(293, 426, 176)
changed img affine  [[ -1.70138884   0.           0.         235.26948547]
 [  0.          -1.69999695   0.         144.93167114]
 [  0.           0.          -1.70138884 -21.65872765]
 [  0.           0.           0.           1.        ]]
changed liver affine  [[ -1.66666663  -0.          -0.         235.26948547]
 [ -0.          -1.66666663  -0.         144.93167114]
 [ -0.           0.          -1.69999695 -21.65872765]
 [  0.           0.           0.           1.        ]]
liver inv  [[ -0.60000001  -0.          -0.         141.16169465]
 [ -0.          -0.60000001  -0.          86.95900476]
 [ -0.          -0.          -0.58823635 -12.7404509 ]
 [  0.           0.           0.          

[293, 12, 33]
[293, 12, 34]
[293, 12, 35]
[293, 12, 36]
[293, 12, 37]
[293, 12, 38]
[293, 12, 39]
[293, 12, 40]
[293, 12, 41]
[293, 12, 42]
[293, 12, 43]
[293, 12, 44]
[293, 12, 45]
[293, 12, 46]
[293, 12, 47]
[293, 12, 48]
[293, 12, 49]
[293, 12, 50]
[293, 12, 51]
[293, 12, 52]
[293, 12, 53]
[293, 12, 54]
[293, 12, 55]
[293, 12, 56]
[293, 12, 57]
[293, 12, 58]
[293, 12, 59]
[293, 12, 60]
[293, 12, 61]
[293, 12, 62]
[293, 12, 63]
[293, 12, 64]
[293, 12, 65]
[293, 12, 66]
[293, 12, 67]
[293, 12, 68]
[293, 12, 69]
[293, 12, 70]
[293, 12, 71]
[293, 12, 72]
[293, 12, 73]
[293, 12, 74]
[293, 12, 75]
[293, 12, 76]
[293, 12, 77]
[293, 12, 78]
[293, 12, 79]
[293, 12, 80]
[293, 12, 81]
[293, 12, 82]
[293, 12, 83]
[293, 12, 84]
[293, 12, 85]
[293, 12, 86]
[293, 12, 87]
[293, 12, 88]
[293, 12, 89]
[293, 12, 90]
[293, 12, 91]
[293, 12, 92]
[293, 12, 93]
[293, 12, 94]
[293, 12, 95]
[293, 12, 96]
[293, 12, 97]
[293, 12, 98]
[293, 12, 99]
[293, 12, 100]
[293, 12, 101]
[293, 12, 102]
[293, 12, 103]
[2

[293, 23, 18]
[293, 23, 19]
[293, 23, 20]
[293, 23, 21]
[293, 23, 22]
[293, 23, 23]
[293, 23, 24]
[293, 23, 25]
[293, 23, 26]
[293, 23, 27]
[293, 23, 28]
[293, 23, 29]
[293, 23, 30]
[293, 23, 31]
[293, 23, 32]
[293, 23, 33]
[293, 23, 34]
[293, 23, 35]
[293, 23, 36]
[293, 23, 37]
[293, 23, 38]
[293, 23, 39]
[293, 23, 40]
[293, 23, 41]
[293, 23, 42]
[293, 23, 43]
[293, 23, 44]
[293, 23, 45]
[293, 23, 46]
[293, 23, 47]
[293, 23, 48]
[293, 23, 49]
[293, 23, 50]
[293, 23, 51]
[293, 23, 52]
[293, 23, 53]
[293, 23, 54]
[293, 23, 55]
[293, 23, 56]
[293, 23, 57]
[293, 23, 58]
[293, 23, 59]
[293, 23, 60]
[293, 23, 61]
[293, 23, 62]
[293, 23, 63]
[293, 23, 64]
[293, 23, 65]
[293, 23, 66]
[293, 23, 67]
[293, 23, 68]
[293, 23, 69]
[293, 23, 70]
[293, 23, 71]
[293, 23, 72]
[293, 23, 73]
[293, 23, 74]
[293, 23, 75]
[293, 23, 76]
[293, 23, 77]
[293, 23, 78]
[293, 23, 79]
[293, 23, 80]
[293, 23, 81]
[293, 23, 82]
[293, 23, 83]
[293, 23, 84]
[293, 23, 85]
[293, 23, 86]
[293, 23, 87]
[293, 23, 88]
[293, 

...

[293, 242, 120]
[293, 242, 121]
[293, 242, 122]
[293, 242, 123]
[293, 242, 124]
[293, 242, 125]
[293, 242, 126]
[293, 242, 127]
[293, 242, 128]
[293, 242, 129]
[293, 242, 130]
[293, 242, 131]
[293, 242, 132]
[293, 242, 133]
[293, 242, 134]
[293, 242, 135]
[293, 242, 136]
[293, 242, 137]
[293, 242, 138]
[293, 242, 139]
[293, 242, 140]
[293, 242, 141]
[293, 242, 142]
[293, 242, 143]
[293, 242, 144]
[293, 242, 145]
[293, 242, 146]
[293, 242, 147]
[293, 242, 148]
[293, 242, 149]
[293, 242, 150]
[293, 242, 151]
[293, 242, 152]
[293, 242, 153]
[293, 242, 154]
[293, 242, 155]
[293, 242, 156]
[293, 242, 157]
[293, 242, 158]
[293, 242, 159]
[293, 242, 160]
[293, 242, 161]
[293, 242, 162]
[293, 242, 163]
[293, 242, 164]
[293, 242, 165]
[293, 242, 166]
[293, 242, 167]
[293, 242, 168]
[293, 242, 169]
[293, 242, 170]
[293, 242, 171]
[293, 242, 172]
[293, 242, 173]
[293, 242, 174]
[293, 242, 175]
[293, 243, 0]
[293, 243, 1]
[293, 243, 2]
[293, 243, 3]
[293, 243, 4]
[293, 243, 5]
[293, 243, 6]
[293, 

[293, 253, 77]
[293, 253, 78]
[293, 253, 79]
[293, 253, 80]
[293, 253, 81]
[293, 253, 82]
[293, 253, 83]
[293, 253, 84]
[293, 253, 85]
[293, 253, 86]
[293, 253, 87]
[293, 253, 88]
[293, 253, 89]
[293, 253, 90]
[293, 253, 91]
[293, 253, 92]
[293, 253, 93]
[293, 253, 94]
[293, 253, 95]
[293, 253, 96]
[293, 253, 97]
[293, 253, 98]
[293, 253, 99]
[293, 253, 100]
[293, 253, 101]
[293, 253, 102]
[293, 253, 103]
[293, 253, 104]
[293, 253, 105]
[293, 253, 106]
[293, 253, 107]
[293, 253, 108]
[293, 253, 109]
[293, 253, 110]
[293, 253, 111]
[293, 253, 112]
[293, 253, 113]
[293, 253, 114]
[293, 253, 115]
[293, 253, 116]
[293, 253, 117]
[293, 253, 118]
[293, 253, 119]
[293, 253, 120]
[293, 253, 121]
[293, 253, 122]
[293, 253, 123]
[293, 253, 124]
[293, 253, 125]
[293, 253, 126]
[293, 253, 127]
[293, 253, 128]
[293, 253, 129]
[293, 253, 130]
[293, 253, 131]
[293, 253, 132]
[293, 253, 133]
[293, 253, 134]
[293, 253, 135]
[293, 253, 136]
[293, 253, 137]
[293, 253, 138]
[293, 253, 139]
[293, 253, 140]

[293, 264, 5]
[293, 264, 6]
[293, 264, 7]
[293, 264, 8]
[293, 264, 9]
[293, 264, 10]
[293, 264, 11]
[293, 264, 12]
[293, 264, 13]
[293, 264, 14]
[293, 264, 15]
[293, 264, 16]
[293, 264, 17]
[293, 264, 18]
[293, 264, 19]
[293, 264, 20]
[293, 264, 21]
[293, 264, 22]
[293, 264, 23]
[293, 264, 24]
[293, 264, 25]
[293, 264, 26]
[293, 264, 27]
[293, 264, 28]
[293, 264, 29]
[293, 264, 30]
[293, 264, 31]
[293, 264, 32]
[293, 264, 33]
[293, 264, 34]
[293, 264, 35]
[293, 264, 36]
[293, 264, 37]
[293, 264, 38]
[293, 264, 39]
[293, 264, 40]
[293, 264, 41]
[293, 264, 42]
[293, 264, 43]
[293, 264, 44]
[293, 264, 45]
[293, 264, 46]
[293, 264, 47]
[293, 264, 48]
[293, 264, 49]
[293, 264, 50]
[293, 264, 51]
[293, 264, 52]
[293, 264, 53]
[293, 264, 54]
[293, 264, 55]
[293, 264, 56]
[293, 264, 57]
[293, 264, 58]
[293, 264, 59]
[293, 264, 60]
[293, 264, 61]
[293, 264, 62]
[293, 264, 63]
[293, 264, 64]
[293, 264, 65]
[293, 264, 66]
[293, 264, 67]
[293, 264, 68]
[293, 264, 69]
[293, 264, 70]
[293, 264, 71]


[293, 273, 135]
[293, 273, 136]
[293, 273, 137]
[293, 273, 138]
[293, 273, 139]
[293, 273, 140]
[293, 273, 141]
[293, 273, 142]
[293, 273, 143]
[293, 273, 144]
[293, 273, 145]
[293, 273, 146]
[293, 273, 147]
[293, 273, 148]
[293, 273, 149]
[293, 273, 150]
[293, 273, 151]
[293, 273, 152]
[293, 273, 153]
[293, 273, 154]
[293, 273, 155]
[293, 273, 156]
[293, 273, 157]
[293, 273, 158]
[293, 273, 159]
[293, 273, 160]
[293, 273, 161]
[293, 273, 162]
[293, 273, 163]
[293, 273, 164]
[293, 273, 165]
[293, 273, 166]
[293, 273, 167]
[293, 273, 168]
[293, 273, 169]
[293, 273, 170]
[293, 273, 171]
[293, 273, 172]
[293, 273, 173]
[293, 273, 174]
[293, 273, 175]
[293, 274, 0]
[293, 274, 1]
[293, 274, 2]
[293, 274, 3]
[293, 274, 4]
[293, 274, 5]
[293, 274, 6]
[293, 274, 7]
[293, 274, 8]
[293, 274, 9]
[293, 274, 10]
[293, 274, 11]
[293, 274, 12]
[293, 274, 13]
[293, 274, 14]
[293, 274, 15]
[293, 274, 16]
[293, 274, 17]
[293, 274, 18]
[293, 274, 19]
[293, 274, 20]
[293, 274, 21]
[293, 274, 22]
[293, 274

[293, 285, 83]
[293, 285, 84]
[293, 285, 85]
[293, 285, 86]
[293, 285, 87]
[293, 285, 88]
[293, 285, 89]
[293, 285, 90]
[293, 285, 91]
[293, 285, 92]
[293, 285, 93]
[293, 285, 94]
[293, 285, 95]
[293, 285, 96]
[293, 285, 97]
[293, 285, 98]
[293, 285, 99]
[293, 285, 100]
[293, 285, 101]
[293, 285, 102]
[293, 285, 103]
[293, 285, 104]
[293, 285, 105]
[293, 285, 106]
[293, 285, 107]
[293, 285, 108]
[293, 285, 109]
[293, 285, 110]
[293, 285, 111]
[293, 285, 112]
[293, 285, 113]
[293, 285, 114]
[293, 285, 115]
[293, 285, 116]
[293, 285, 117]
[293, 285, 118]
[293, 285, 119]
[293, 285, 120]
[293, 285, 121]
[293, 285, 122]
[293, 285, 123]
[293, 285, 124]
[293, 285, 125]
[293, 285, 126]
[293, 285, 127]
[293, 285, 128]
[293, 285, 129]
[293, 285, 130]
[293, 285, 131]
[293, 285, 132]
[293, 285, 133]
[293, 285, 134]
[293, 285, 135]
[293, 285, 136]
[293, 285, 137]
[293, 285, 138]
[293, 285, 139]
[293, 285, 140]
[293, 285, 141]
[293, 285, 142]
[293, 285, 143]
[293, 285, 144]
[293, 285, 145]
[293, 285

In [52]:
print(np.max(new_liver))
new_img_nii = nibabel.MGHImage(new_liver, img_affine, img_h)
new_file_path = os.path.join(seg_dir, 'cropped_spleen2.nii.gz')
nibabel.save(new_img_nii, new_file_path)

1.0
